In [1]:
# coding: utf-8
import os

import pandas as pd
import numpy as np
import scipy
import lightgbm as lgb
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.pyplot as plt
import seaborn as sns

from mydatools.plot import plot_multiclass_feature_dist
from mydatools.features_analyze import get_top_k_corr

% matplotlib inline

## Config

In [2]:
trn_path = './data/input/application_train.csv'
tst_path = './data/input/application_test.csv'
id_col = 'SK_ID_CURR'
label_col = 'TARGET'

submission_path = './data/output/submission/submission.csv'
output_id_col = id_col
output_label_col = label_col

## Load Data

In [3]:
trn_df = pd.read_csv(trn_path)
trn_df['ds_type'] = 'trn'
trn_df[label_col] = trn_df[label_col].astype('int')

tst_df = pd.read_csv(tst_path)
tst_df['ds_type'] = 'tst'

full_df = pd.concat([trn_df, tst_df])

del(trn_df)
del(tst_df)

full_df.head()

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,...,TOTALAREA_MODE,WALLSMATERIAL_MODE,WEEKDAY_APPR_PROCESS_START,YEARS_BEGINEXPLUATATION_AVG,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_AVG,YEARS_BUILD_MEDI,YEARS_BUILD_MODE,ds_type
0,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0149,"Stone, brick",WEDNESDAY,0.9722,0.9722,0.9722,0.6192,0.6243,0.6341,trn
1,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0714,Block,MONDAY,0.9851,0.9851,0.9851,0.7960,0.7987,0.8040,trn
2,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,MONDAY,NaN,NaN,NaN,NaN,NaN,NaN,trn
3,29686.5,312682.5,297000.0,135000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,WEDNESDAY,NaN,NaN,NaN,NaN,NaN,NaN,trn
4,21865.5,513000.0,513000.0,121500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,THURSDAY,NaN,NaN,NaN,NaN,NaN,NaN,trn


In [4]:
# dataset type
is_trn = full_df['ds_type'] == 'trn'
is_tst = full_df['ds_type'] == 'tst'

## Features

In [5]:
feature_columns = []

def add_features(features):
    if not isinstance(features, list):
        features = [features]
    global feature_columns
    feature_columns.extend([f for f in features if f not in feature_columns])

**numerical_features**

In [6]:
numerical_features = full_df.dtypes[full_df.dtypes != 'object'].index.tolist()
numerical_features = [c for c in numerical_features if c not in [id_col, label_col, 'ds_type']]
add_features(numerical_features)

**categorical_features**

In [7]:
categorical_features = full_df.dtypes[full_df.dtypes == 'object'].index.tolist()
full_df[categorical_features].nunique()

CODE_GENDER                    3
EMERGENCYSTATE_MODE            2
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
FONDKAPREMONT_MODE             4
HOUSETYPE_MODE                 3
NAME_CONTRACT_TYPE             2
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
NAME_INCOME_TYPE               8
NAME_TYPE_SUITE                7
OCCUPATION_TYPE               18
ORGANIZATION_TYPE             58
WALLSMATERIAL_MODE             7
WEEKDAY_APPR_PROCESS_START     7
ds_type                        2
dtype: int64

In [8]:
categorical_features = full_df.dtypes[full_df.dtypes == 'object'].index.tolist()
categorical_features = categorical_features[:-1] # 为了去掉ds_type
# 去掉分类数太多
# categorical_features = full_df[categorical_features].columns[full_df[categorical_features].nunique() <= 10]

# get dummies
full_df = pd.get_dummies(full_df, columns=categorical_features, prefix_sep=':')

# add features
new_features = full_df.columns.tolist()[full_df.columns.tolist().index('ds_type') + 1:]
add_features(new_features)

### add extra data

In [9]:
def summary_extra_data(df, suffix):
    # 分类特征get_dummies
    df = pd.get_dummies(df, columns=df.columns[df.dtypes == 'object'].tolist())
    df = df.fillna(0)

    # 求平均
    df = pd.concat([
        df.groupby('SK_ID_CURR').mean(),
        df.groupby('SK_ID_CURR').size().rename('cnt'),
    ], axis=1)

    # 给特征加上前缀
    df.columns = [suffix+f for f in df.columns.tolist()]

    return df

In [10]:
# bureau = pd.read_csv('./data/input/bureau.csv')
# test_df = bureau.iloc[:100, :5]
# test_df.head()

# summary_extra_data(test_df, 'test')

**bureau.csv**

In [11]:
bureau = pd.read_csv('./data/input/bureau.csv')
bureau = bureau.drop('SK_ID_BUREAU', axis=1)

bureau = summary_extra_data(bureau, 'bureau_')

full_df = full_df.join(bureau, on='SK_ID_CURR')
add_features(bureau.columns.tolist())

**previous_application.csv**

In [12]:
prevapp = pd.read_csv('./data/input/previous_application.csv')
prevapp = prevapp.drop('SK_ID_PREV', axis=1)

prevapp = summary_extra_data(prevapp, 'prevapp_')

full_df = full_df.join(prevapp, on='SK_ID_CURR')
add_features(prevapp.columns.tolist())

## LightGBM

In [13]:
trn_df = full_df[is_trn]
tst_df = full_df[is_tst]

X = trn_df[feature_columns]
y = trn_df[label_col]
X_trn, X_val, y_trn, y_val = train_test_split(X, y, test_size=0.20, random_state=20180521)

X_tst = tst_df[feature_columns]

d_trn = lgb.Dataset(X_trn.values, y_trn)
d_val = lgb.Dataset(X_val.values, y_val)

In [14]:
params = {
    'application': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'max_depth': 4,
    'num_leaves': 20,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.5,
    'bagging_freq': 1,
    'is_unbalance': True,
    'seed': 0,
    'reg_alpha': 0.1,
}
bst = lgb.train(params, d_trn, 500, valid_sets=[d_trn, d_val], early_stopping_rounds=20)

[1]	training's auc: 0.702702	valid_1's auc: 0.704245
Training until validation scores don't improve for 20 rounds.
[2]	training's auc: 0.711497	valid_1's auc: 0.714016
[3]	training's auc: 0.717769	valid_1's auc: 0.720827
[4]	training's auc: 0.720255	valid_1's auc: 0.725001
[5]	training's auc: 0.721526	valid_1's auc: 0.726042
[6]	training's auc: 0.721904	valid_1's auc: 0.727198
[7]	training's auc: 0.722892	valid_1's auc: 0.727866
[8]	training's auc: 0.723546	valid_1's auc: 0.728359
[9]	training's auc: 0.72369	valid_1's auc: 0.728689
[10]	training's auc: 0.724503	valid_1's auc: 0.728962
[11]	training's auc: 0.725095	valid_1's auc: 0.7292
[12]	training's auc: 0.725941	valid_1's auc: 0.729892
[13]	training's auc: 0.726315	valid_1's auc: 0.73012
[14]	training's auc: 0.727056	valid_1's auc: 0.731014
[15]	training's auc: 0.728241	valid_1's auc: 0.731999
[16]	training's auc: 0.729067	valid_1's auc: 0.732746
[17]	training's auc: 0.729612	valid_1's auc: 0.733124
[18]	training's auc: 0.72998	vali

[153]	training's auc: 0.780322	valid_1's auc: 0.771019
[154]	training's auc: 0.780443	valid_1's auc: 0.771077
[155]	training's auc: 0.78056	valid_1's auc: 0.771144
[156]	training's auc: 0.780658	valid_1's auc: 0.771209
[157]	training's auc: 0.780769	valid_1's auc: 0.771279
[158]	training's auc: 0.780917	valid_1's auc: 0.771284
[159]	training's auc: 0.781048	valid_1's auc: 0.771317
[160]	training's auc: 0.781149	valid_1's auc: 0.771358
[161]	training's auc: 0.781278	valid_1's auc: 0.771386
[162]	training's auc: 0.781522	valid_1's auc: 0.771494
[163]	training's auc: 0.781645	valid_1's auc: 0.771582
[164]	training's auc: 0.781784	valid_1's auc: 0.771628
[165]	training's auc: 0.781935	valid_1's auc: 0.771711
[166]	training's auc: 0.782108	valid_1's auc: 0.77184
[167]	training's auc: 0.78222	valid_1's auc: 0.771917
[168]	training's auc: 0.782328	valid_1's auc: 0.771949
[169]	training's auc: 0.782469	valid_1's auc: 0.771965
[170]	training's auc: 0.782677	valid_1's auc: 0.772093
[171]	trainin

[304]	training's auc: 0.798189	valid_1's auc: 0.776997
[305]	training's auc: 0.798266	valid_1's auc: 0.776978
[306]	training's auc: 0.798345	valid_1's auc: 0.776974
[307]	training's auc: 0.798407	valid_1's auc: 0.776957
[308]	training's auc: 0.798515	valid_1's auc: 0.776959
[309]	training's auc: 0.798607	valid_1's auc: 0.776979
[310]	training's auc: 0.798698	valid_1's auc: 0.777
[311]	training's auc: 0.798766	valid_1's auc: 0.777009
[312]	training's auc: 0.798978	valid_1's auc: 0.777043
[313]	training's auc: 0.799085	valid_1's auc: 0.77711
[314]	training's auc: 0.799182	valid_1's auc: 0.777164
[315]	training's auc: 0.799287	valid_1's auc: 0.777202
[316]	training's auc: 0.799415	valid_1's auc: 0.777199
[317]	training's auc: 0.799507	valid_1's auc: 0.777196
[318]	training's auc: 0.799579	valid_1's auc: 0.777182
[319]	training's auc: 0.799625	valid_1's auc: 0.777147
[320]	training's auc: 0.799747	valid_1's auc: 0.777091
[321]	training's auc: 0.799868	valid_1's auc: 0.777154
[322]	training

[455]	training's auc: 0.811032	valid_1's auc: 0.778772
[456]	training's auc: 0.811118	valid_1's auc: 0.778796
[457]	training's auc: 0.811177	valid_1's auc: 0.778776
[458]	training's auc: 0.811247	valid_1's auc: 0.778796
[459]	training's auc: 0.811273	valid_1's auc: 0.778767
[460]	training's auc: 0.811312	valid_1's auc: 0.778763
[461]	training's auc: 0.811388	valid_1's auc: 0.778774
[462]	training's auc: 0.81143	valid_1's auc: 0.778772
[463]	training's auc: 0.811501	valid_1's auc: 0.778778
[464]	training's auc: 0.81159	valid_1's auc: 0.778874
[465]	training's auc: 0.811665	valid_1's auc: 0.778878
[466]	training's auc: 0.81174	valid_1's auc: 0.778826
[467]	training's auc: 0.811777	valid_1's auc: 0.778802
[468]	training's auc: 0.811874	valid_1's auc: 0.778784
[469]	training's auc: 0.811946	valid_1's auc: 0.778747
[470]	training's auc: 0.812051	valid_1's auc: 0.77871
[471]	training's auc: 0.812114	valid_1's auc: 0.778718
[472]	training's auc: 0.812205	valid_1's auc: 0.778726
[473]	training

In [15]:
# bst = lgb.train(params, trn_lgb, 1000, valid_sets=[trn_lgb, val_lgb], early_stopping_rounds=20,
#                 init_model=bst, 
#                 learning_rates=lambda iter: 0.1 * (0.99 ** iter))

In [16]:
imp_df = pd.DataFrame([bst.feature_importance()], columns=feature_columns, index=['importance']).T.sort_values(by='importance', ascending=False)
imp_df

,importance
EXT_SOURCE_3,309
EXT_SOURCE_2,279
EXT_SOURCE_1,253
DAYS_BIRTH,201
prevapp_CNT_PAYMENT,153
AMT_CREDIT,146
DAYS_EMPLOYED,136
AMT_ANNUITY,131
AMT_GOODS_PRICE,119
DAYS_ID_PUBLISH,101


## predict

In [17]:
res_df = pd.DataFrame(bst.predict(X_tst), columns=[output_label_col])
res_df[output_id_col] = tst_df[output_id_col].values
res_df[[output_id_col, output_label_col]].to_csv(submission_path, index=False)